In [2]:
import pandas as pd
from minimal_rules import find_minimal_rules, generate_prolog_rule
from rough_sets import find_reducts

In [5]:
csv_path = r"..\baza_wiedzy\csv\nieruchomosci.csv"
output_path = r"..\reguly\reguly_minimalne.pl"

1. Find reducts

In [21]:
sample_data = pd.read_csv(csv_path, sep=',')

# keep only 'Standard', 'Typ', 'Lokalizacja', 'Winda', 'Parking', 'TypParking', 'Opis' columns ('Opis' column is redundant)
sample_data = sample_data[['Standard', 'Typ', 'Lokalizacja', 'Winda', 'Parking', 'TypParking', 'Opis']]

# Analyze the sample dataset
results = find_reducts(sample_data, 'Standard')

print("Analysis Results:")
print(f"Core attributes: {results['core_attributes']}")
print(f"Number of reducts found: {results['number_of_reducts']}")
print(f"All reducts: {results['all_reducts']}")
print(f"Dataset is consistent: {results['is_consistent']}")
print(f"Full dependency degree: {results['full_dependency_degree']:.3f}")
print()

print("Reduct details:")
for reduct_name, details in results['reduct_details'].items():
    print(f"  {reduct_name}: {details['attributes']} "
            f"(dependency: {details['dependency_degree']:.3f})")

Analysis Results:
Core attributes: ['Typ']
Number of reducts found: 2
All reducts: [['Typ', 'Lokalizacja'], ['Typ', 'TypParking']]
Dataset is consistent: True
Full dependency degree: 1.000

Reduct details:
  reduct_1: ['Typ', 'Lokalizacja'] (dependency: 1.000)
  reduct_2: ['Typ', 'TypParking'] (dependency: 1.000)


2. Generate minimal rules for 'Standard' column

In [22]:
df = pd.read_csv(csv_path, sep=',')

first_reduct = next(iter(results['reduct_details'].values()))
print(f"Generating minimal rules using the reduct: {first_reduct['attributes']}")

target_column = 'Standard'
prolog_rules = []
for target_value in ['luksusowy', 'wysoki', 'niski', 'sredni']:
    print(f"\nFinding minimal rules for '{target_value}'")
    minimal_rules = find_minimal_rules(df, first_reduct['attributes'], target_column, target_value, min_accuracy=0.7)
    
    best_rule = max(minimal_rules, key=lambda x: x[1])
    print(f"Best rule for target value '{target_value}': {best_rule}")
    
    rule = generate_prolog_rule(best_rule, 'Nieruchomosc', target_column, target_value)
    prolog_rules.append(rule)
    
with open(output_path, 'w') as f:
    f.write("% autogenerated using generate_minimal_rules.ipynb\n\n")
    for rule in prolog_rules:
        f.write(rule + '\n\n')
print(f"Prolog rules saved to {output_path}")

Generating minimal rules using the reduct: ['Typ', 'Lokalizacja']

Finding minimal rules for 'luksusowy'


Combining 2 rules: 100%|██████████| 10/10 [00:00<00:00, 1910.24it/s]


Best rule for target value 'luksusowy': ([('Typ', '=', 'dom')], 1.0)

Finding minimal rules for 'wysoki'


Combining 2 rules: 100%|██████████| 10/10 [00:00<00:00, 1863.97it/s]


Best rule for target value 'wysoki': ([('Typ', '=', 'mieszkanie'), ('Lokalizacja', '=', 'centrum')], 1.0)

Finding minimal rules for 'niski'


Combining 2 rules: 100%|██████████| 10/10 [00:00<00:00, 1962.61it/s]


Best rule for target value 'niski': ([('Lokalizacja', '=', 'przedmiescia')], 1.0)

Finding minimal rules for 'sredni'


Combining 2 rules: 100%|██████████| 10/10 [00:00<00:00, 2173.44it/s]

Best rule for target value 'sredni': ([('Lokalizacja', '=', 'poza centrum')], 1.0)
Prolog rules saved to ..\reguly\reguly_minimalne.pl
